In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kaggle-pog-series-s01e04/sample_submission.csv
/kaggle/input/kaggle-pog-series-s01e04/train_detailed.csv
/kaggle/input/kaggle-pog-series-s01e04/raw_health_export.xml
/kaggle/input/kaggle-pog-series-s01e04/train.csv
/kaggle/input/kaggle-pog-series-s01e04/test.csv


### notes

In [2]:
from fastai.tabular.all import *

### Handle large xml files.

In [3]:
raw_health = Path("/kaggle/input/kaggle-pog-series-s01e04/raw_health_export.xml")

### Convert xml to csv

In [4]:
# import important modules
import csv
import xml.etree.ElementTree as ET

# open the input file
with open(raw_health, 'r') as f:

    # parse the XML data
    health_data = ET.parse(f)

    # open the output file for writing
    with open('raw_health_exp.csv', 'w', newline='') as csv_file:

        # create a CSV writer object
        csv_writer = csv.writer(csv_file)

        # write the header row
        csv_writer.writerow(['type', 'sourceName', 'sourceVersion', 'unit', 'creationDate', 'startDate', 'endDate', 'value'])

        # iterate over each Record element and extract the relevant data
        for i, record in enumerate(health_data.findall('./Record')):
            try:
                # extract the relevant data from the Record element
                r_type = record.get('type')
                source_name = record.get('sourceName')
                source_version = record.get('sourceVersion')
                unit = record.get('unit')
                creation_date = record.get('creationDate')
                start_date = record.get('startDate')
                end_date = record.get('endDate')
                value = record.get('value')

                # write the extracted data to the output file as a row
                csv_writer.writerow([r_type, source_name, source_version, unit, creation_date, start_date, end_date, value])
            except:
                # if an error occurs during processing, skip this record and log an error message
                print(f"Error processing record {i}")
                continue


In [5]:
rh_csv="/kaggle/working/raw_health_exp.csv"
rh_df=pd.read_csv(rh_csv,low_memory=False)
rh_df

,type,sourceName,sourceVersion,unit,creationDate,startDate,endDate,value
0,HKQuantityTypeIdentifierDietaryWater,Sync Solver,25,mL,2016-06-20 13:52:52 -0400,2015-07-17 00:00:00 -0400,2015-07-17 23:59:59 -0400,709.765
1,HKQuantityTypeIdentifierBodyMassIndex,Sync Solver,NaN,count,2015-04-19 09:42:10 -0400,2015-01-17 01:00:00 -0400,2015-01-18 00:59:59 -0400,23.5981
2,HKQuantityTypeIdentifierBodyMassIndex,Sync Solver,NaN,count,2015-04-19 09:42:10 -0400,2013-08-18 00:00:00 -0400,2013-08-18 23:59:59 -0400,23.5981
3,HKQuantityTypeIdentifierBodyMassIndex,Sync Solver,NaN,count,2015-04-19 09:42:10 -0400,2013-05-19 00:00:00 -0400,2013-05-19 23:59:59 -0400,23.5981
4,HKQuantityTypeIdentifierBodyMassIndex,Sync Solver,NaN,count,2015-04-19 09:42:10 -0400,2013-04-25 00:00:00 -0400,2013-04-25 23:59:59 -0400,23.5981
...,...,...,...,...,...,...,...,...
2795073,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,Rob’s Apple Watch,9.3.1,ms,2023-03-17 10:10:05 -0400,2023-03-17 10:09:04 -0400,2023-03-17 10:10:03 -0400,52.3319
2795074,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,Rob’s Apple Watch,9.3.1,ms,2023-03-17 18:54:31 -0400,2023-03-17 18:53:30 -0400,2023-03-17 18:54:29 -0400,47.957
2795075,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,Rob’s Apple Watch,9.3.1,ms,2023-03-17 19:55:29 -0400,2023-03-17 19:54:28 -0400,2023-03-17 19:55:28 -0400,126.735
2795076,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,Rob’s Apple Watch,9.3.1,ms,2023-03-17 21:55:26 -0400,2023-03-17 21:54:24 -0400,2023-03-17 21:55:24 -0400,54.4332


In [6]:
sam_sub=Path('/kaggle/input/kaggle-pog-series-s01e04/sample_submission.csv')
test = Path('/kaggle/input/kaggle-pog-series-s01e04/test.csv')
train=Path('/kaggle/input/kaggle-pog-series-s01e04/train.csv')
train_det=Path('/kaggle/input/kaggle-pog-series-s01e04/train_detailed.csv')

In [7]:
ss_df=pd.read_csv(sam_sub)
ss_df

,date,sleep_hours
0,2022-01-01,1
1,2022-01-02,1
2,2022-01-03,1
3,2022-01-04,1
4,2022-01-05,1
...,...,...
414,2023-03-12,1
415,2023-03-13,1
416,2023-03-14,1
417,2023-03-15,1


In [8]:
tr_df=pd.read_csv(train)
tr_df

,date,sleep_hours
0,2015-02-19,6.400000
1,2015-02-20,7.583333
2,2015-02-21,6.350000
3,2015-02-22,6.500000
4,2015-02-23,8.916667
...,...,...
2349,2021-12-25,7.933333
2350,2021-12-26,3.850000
2351,2021-12-29,6.175000
2352,2021-12-30,5.158333


- **Remove creation date**
- **Its same for each source name.**

- **Here we are trying to organize data & time into separate columns .**

In [9]:
# convert the datetime column to datetime datatype
rh_df['endDate'] = pd.to_datetime(rh_df['endDate'])

# add columns for date and time
rh_df['Edate'] = rh_df['endDate'].dt.strftime('%Y-%m-%d')
rh_df['Etime'] = rh_df['endDate'].dt.strftime('%H:%M:%S')

In [10]:
rh_df['startDate'] = pd.to_datetime(rh_df['startDate'])

# add columns for date and time
rh_df['Sdate'] = rh_df['startDate'].dt.strftime('%Y-%m-%d')
rh_df['Stime'] = rh_df['startDate'].dt.strftime('%H:%M:%S')

In [11]:
rh_df=rh_df.drop(['startDate','endDate'],axis=1)

In [12]:
rh_df

,type,sourceName,sourceVersion,unit,creationDate,value,Edate,Etime,Sdate,Stime
0,HKQuantityTypeIdentifierDietaryWater,Sync Solver,25,mL,2016-06-20 13:52:52 -0400,709.765,2015-07-17,23:59:59,2015-07-17,00:00:00
1,HKQuantityTypeIdentifierBodyMassIndex,Sync Solver,NaN,count,2015-04-19 09:42:10 -0400,23.5981,2015-01-18,00:59:59,2015-01-17,01:00:00
2,HKQuantityTypeIdentifierBodyMassIndex,Sync Solver,NaN,count,2015-04-19 09:42:10 -0400,23.5981,2013-08-18,23:59:59,2013-08-18,00:00:00
3,HKQuantityTypeIdentifierBodyMassIndex,Sync Solver,NaN,count,2015-04-19 09:42:10 -0400,23.5981,2013-05-19,23:59:59,2013-05-19,00:00:00
4,HKQuantityTypeIdentifierBodyMassIndex,Sync Solver,NaN,count,2015-04-19 09:42:10 -0400,23.5981,2013-04-25,23:59:59,2013-04-25,00:00:00
...,...,...,...,...,...,...,...,...,...,...
2795073,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,Rob’s Apple Watch,9.3.1,ms,2023-03-17 10:10:05 -0400,52.3319,2023-03-17,10:10:03,2023-03-17,10:09:04
2795074,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,Rob’s Apple Watch,9.3.1,ms,2023-03-17 18:54:31 -0400,47.957,2023-03-17,18:54:29,2023-03-17,18:53:30
2795075,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,Rob’s Apple Watch,9.3.1,ms,2023-03-17 19:55:29 -0400,126.735,2023-03-17,19:55:28,2023-03-17,19:54:28
2795076,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,Rob’s Apple Watch,9.3.1,ms,2023-03-17 21:55:26 -0400,54.4332,2023-03-17,21:55:24,2023-03-17,21:54:24


In [14]:
 # merge the dataframes based on the date column, keeping only rows where the dates match
merged_df = pd.merge(rh_df,tr_df, left_on='Sdate',right_on='date', how='inner')



In [15]:
merged_df

,type,sourceName,sourceVersion,unit,creationDate,value,Edate,Etime,Sdate,Stime,date,sleep_hours
0,HKQuantityTypeIdentifierDietaryWater,Sync Solver,25,mL,2016-06-20 13:52:52 -0400,709.765,2015-07-17,23:59:59,2015-07-17,00:00:00,2015-07-17,6.900
1,HKQuantityTypeIdentifierBodyMassIndex,Sync Solver,NaN,count,2015-08-11 17:53:16 -0400,23.4358,2015-07-17,23:59:59,2015-07-17,00:00:00,2015-07-17,6.900
2,HKQuantityTypeIdentifierBodyMass,Sync Solver,NaN,lb,2015-08-11 17:53:12 -0400,187.501,2015-07-17,23:59:59,2015-07-17,00:00:00,2015-07-17,6.900
3,HKQuantityTypeIdentifierStepCount,Robs iPhone,NaN,count,2015-07-17 02:50:45 -0400,81,2015-07-17,01:27:29,2015-07-17,01:21:56,2015-07-17,6.900
4,HKQuantityTypeIdentifierStepCount,Robs iPhone,NaN,count,2015-07-17 02:50:45 -0400,10,2015-07-17,01:27:49,2015-07-17,01:27:29,2015-07-17,6.900
...,...,...,...,...,...,...,...,...,...,...,...,...
1188100,HKQuantityTypeIdentifierStairDescentSpeed,Rob’s Apple Watch,2665.0.8,ft/s,2021-12-29 23:34:43 -0400,1.41973,2021-12-29,23:34:03,2021-12-29,23:33:58,2021-12-29,6.175
1188101,HKCategoryTypeIdentifierAppleStandHour,Rob’s Apple Watch,8.3,NaN,2021-12-29 22:13:02 -0400,HKCategoryValueAppleStandHourIdle,2021-12-29,21:00:00,2021-12-29,20:00:00,2021-12-29,6.175
1188102,HKCategoryTypeIdentifierAppleStandHour,Rob’s Apple Watch,8.3,NaN,2021-12-29 22:13:03 -0400,HKCategoryValueAppleStandHourIdle,2021-12-29,22:00:00,2021-12-29,21:00:00,2021-12-29,6.175
1188103,HKCategoryTypeIdentifierAppleStandHour,Rob’s Apple Watch,8.3,NaN,2021-12-29 23:00:11 -0400,HKCategoryValueAppleStandHourIdle,2021-12-29,23:00:00,2021-12-29,22:00:00,2021-12-29,6.175


In [16]:
rh_df=merged_df.drop('date',axis=1)

In [17]:
rh_df

,type,sourceName,sourceVersion,unit,creationDate,value,Edate,Etime,Sdate,Stime,sleep_hours
0,HKQuantityTypeIdentifierDietaryWater,Sync Solver,25,mL,2016-06-20 13:52:52 -0400,709.765,2015-07-17,23:59:59,2015-07-17,00:00:00,6.900
1,HKQuantityTypeIdentifierBodyMassIndex,Sync Solver,NaN,count,2015-08-11 17:53:16 -0400,23.4358,2015-07-17,23:59:59,2015-07-17,00:00:00,6.900
2,HKQuantityTypeIdentifierBodyMass,Sync Solver,NaN,lb,2015-08-11 17:53:12 -0400,187.501,2015-07-17,23:59:59,2015-07-17,00:00:00,6.900
3,HKQuantityTypeIdentifierStepCount,Robs iPhone,NaN,count,2015-07-17 02:50:45 -0400,81,2015-07-17,01:27:29,2015-07-17,01:21:56,6.900
4,HKQuantityTypeIdentifierStepCount,Robs iPhone,NaN,count,2015-07-17 02:50:45 -0400,10,2015-07-17,01:27:49,2015-07-17,01:27:29,6.900
...,...,...,...,...,...,...,...,...,...,...,...
1188100,HKQuantityTypeIdentifierStairDescentSpeed,Rob’s Apple Watch,2665.0.8,ft/s,2021-12-29 23:34:43 -0400,1.41973,2021-12-29,23:34:03,2021-12-29,23:33:58,6.175
1188101,HKCategoryTypeIdentifierAppleStandHour,Rob’s Apple Watch,8.3,NaN,2021-12-29 22:13:02 -0400,HKCategoryValueAppleStandHourIdle,2021-12-29,21:00:00,2021-12-29,20:00:00,6.175
1188102,HKCategoryTypeIdentifierAppleStandHour,Rob’s Apple Watch,8.3,NaN,2021-12-29 22:13:03 -0400,HKCategoryValueAppleStandHourIdle,2021-12-29,22:00:00,2021-12-29,21:00:00,6.175
1188103,HKCategoryTypeIdentifierAppleStandHour,Rob’s Apple Watch,8.3,NaN,2021-12-29 23:00:11 -0400,HKCategoryValueAppleStandHourIdle,2021-12-29,23:00:00,2021-12-29,22:00:00,6.175


In [18]:
add_datepart(rh_df,'Sdate')
add_datepart(rh_df,'Stime')
add_datepart(rh_df,'Edate')
add_datepart(rh_df,'Etime')

rh_df=rh_df.drop(["StimeElapsed","EtimeElapsed"],axis=1)

- **Two possible sol - remove rows with val string or repace them**

In [19]:
rh_df

,type,sourceName,sourceVersion,unit,creationDate,value,sleep_hours,SYear,SMonth,SWeek,...,EtimeWeek,EtimeDay,EtimeDayofweek,EtimeDayofyear,EtimeIs_month_end,EtimeIs_month_start,EtimeIs_quarter_end,EtimeIs_quarter_start,EtimeIs_year_end,EtimeIs_year_start
0,HKQuantityTypeIdentifierDietaryWater,Sync Solver,25,mL,2016-06-20 13:52:52 -0400,709.765,6.900,2015,7,29,...,37,14,3,257,False,False,False,False,False,False
1,HKQuantityTypeIdentifierBodyMassIndex,Sync Solver,NaN,count,2015-08-11 17:53:16 -0400,23.4358,6.900,2015,7,29,...,37,14,3,257,False,False,False,False,False,False
2,HKQuantityTypeIdentifierBodyMass,Sync Solver,NaN,lb,2015-08-11 17:53:12 -0400,187.501,6.900,2015,7,29,...,37,14,3,257,False,False,False,False,False,False
3,HKQuantityTypeIdentifierStepCount,Robs iPhone,NaN,count,2015-07-17 02:50:45 -0400,81,6.900,2015,7,29,...,37,14,3,257,False,False,False,False,False,False
4,HKQuantityTypeIdentifierStepCount,Robs iPhone,NaN,count,2015-07-17 02:50:45 -0400,10,6.900,2015,7,29,...,37,14,3,257,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1188100,HKQuantityTypeIdentifierStairDescentSpeed,Rob’s Apple Watch,2665.0.8,ft/s,2021-12-29 23:34:43 -0400,1.41973,6.175,2021,12,52,...,37,14,3,257,False,False,False,False,False,False
1188101,HKCategoryTypeIdentifierAppleStandHour,Rob’s Apple Watch,8.3,NaN,2021-12-29 22:13:02 -0400,HKCategoryValueAppleStandHourIdle,6.175,2021,12,52,...,37,14,3,257,False,False,False,False,False,False
1188102,HKCategoryTypeIdentifierAppleStandHour,Rob’s Apple Watch,8.3,NaN,2021-12-29 22:13:03 -0400,HKCategoryValueAppleStandHourIdle,6.175,2021,12,52,...,37,14,3,257,False,False,False,False,False,False
1188103,HKCategoryTypeIdentifierAppleStandHour,Rob’s Apple Watch,8.3,NaN,2021-12-29 23:00:11 -0400,HKCategoryValueAppleStandHourIdle,6.175,2021,12,52,...,37,14,3,257,False,False,False,False,False,False


In [26]:
to_remove = ['HKCategoryValueAppleStandHourIdle','HKCategoryValueAppleStandHourStood',"HKCategoryValueEnvironmentalAudioExposureEventMomentaryLimit","HKCategoryValueHeadphoneAudioExposureEventSevenDayLimit"]
mask = rh_df['value'].isin(to_remove)
rh_df = rh_df[~mask]

In [27]:
rh_df=rh_df.drop(["sourceName","sourceVersion","unit","creationDate"],axis=1)

KeyError: "['sourceName' 'sourceVersion' 'unit' 'creationDate'] not found in axis"

In [22]:
rh_df

,type,value,sleep_hours,SYear,SMonth,SWeek,SDay,SDayofweek,SDayofyear,SIs_month_end,...,EtimeWeek,EtimeDay,EtimeDayofweek,EtimeDayofyear,EtimeIs_month_end,EtimeIs_month_start,EtimeIs_quarter_end,EtimeIs_quarter_start,EtimeIs_year_end,EtimeIs_year_start
0,HKQuantityTypeIdentifierDietaryWater,709.765,6.900,2015,7,29,17,4,198,False,...,37,14,3,257,False,False,False,False,False,False
1,HKQuantityTypeIdentifierBodyMassIndex,23.4358,6.900,2015,7,29,17,4,198,False,...,37,14,3,257,False,False,False,False,False,False
2,HKQuantityTypeIdentifierBodyMass,187.501,6.900,2015,7,29,17,4,198,False,...,37,14,3,257,False,False,False,False,False,False
3,HKQuantityTypeIdentifierStepCount,81,6.900,2015,7,29,17,4,198,False,...,37,14,3,257,False,False,False,False,False,False
4,HKQuantityTypeIdentifierStepCount,10,6.900,2015,7,29,17,4,198,False,...,37,14,3,257,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1188096,HKQuantityTypeIdentifierWalkingAsymmetryPercentage,0,6.175,2021,12,52,29,2,363,False,...,37,14,3,257,False,False,False,False,False,False
1188097,HKQuantityTypeIdentifierWalkingAsymmetryPercentage,0.06,6.175,2021,12,52,29,2,363,False,...,37,14,3,257,False,False,False,False,False,False
1188098,HKQuantityTypeIdentifierStairAscentSpeed,1.33482,6.175,2021,12,52,29,2,363,False,...,37,14,3,257,False,False,False,False,False,False
1188099,HKQuantityTypeIdentifierStairAscentSpeed,1.21347,6.175,2021,12,52,29,2,363,False,...,37,14,3,257,False,False,False,False,False,False


In [23]:
rh_df["value"] = rh_df["value"].astype(float)


In [24]:
rh_df["value"].dtype

dtype('float64')

In [25]:
print(rh_df.dtypes)

type                      object
value                    float64
sleep_hours              float64
SYear                      int64
SMonth                     int64
SWeek                      int64
SDay                       int64
SDayofweek                 int64
SDayofyear                 int64
SIs_month_end               bool
SIs_month_start             bool
SIs_quarter_end             bool
SIs_quarter_start           bool
SIs_year_end                bool
SIs_year_start              bool
SElapsed                 float64
StimeYear                  int64
StimeMonth                 int64
StimeWeek                  int64
StimeDay                   int64
StimeDayofweek             int64
StimeDayofyear             int64
StimeIs_month_end           bool
StimeIs_month_start         bool
StimeIs_quarter_end         bool
StimeIs_quarter_start       bool
StimeIs_year_end            bool
StimeIs_year_start          bool
EYear                      int64
EMonth                     int64
EWeek     

In [ ]:
EtimeIs_month_end          
EtimeIs_month_start         EtimeIs_quarter_end         
EtimeIs_quarter_start       
EtimeIs_year_end            
EtimeIs_year_start 

EIs_month_end               
EIs_month_start             
EIs_quarter_end             
EIs_quarter_start         EIs_year_end                
EIs_year_start

StimeIs_month_end           
StimeIs_month_start         
StimeIs_quarter_end         
StimeIs_quarter_start       
StimeIs_year_end            
StimeIs_year_start  

SIs_month_end               
SIs_month_start             
SIs_quarter_end             
SIs_quarter_start           
SIs_year_end                
SIs_year_start   

In [34]:
df_encoded = pd.get_dummies(rh_df, columns=["SIs_month_end","EtimeIs_month_end" ,"EtimeIs_month_start", "EtimeIs_quarter_end", "EtimeIs_quarter_start","EtimeIs_year_end" ,"EtimeIs_year_start"])

In [36]:
df_encoded = pd.get_dummies(df_encoded, columns=['SIs_month_start'])EtimeIs_month_end          



In [41]:
df_encoded_data = pd.get_dummies(df_encoded ,columns=['EIs_month_end','EIs_month_start', 'EIs_quarter_end', 'EIs_quarter_start' ,'EIs_year_end', 'EIs_year_start', 'StimeIs_month_start', 'StimeIs_quarter_end' ,'StimeIs_quarter_start', 'StimeIs_year_end' ,'StimeIs_year_start','SIs_quarter_end', 'SIs_quarter_start' , 'SIs_year_end', 'SIs_year_start'])

In [42]:
df_encoded_data.head()

,type,value,sleep_hours,SYear,SMonth,SWeek,SDay,SDayofweek,SDayofyear,SElapsed,...,StimeIs_year_end_False,StimeIs_year_start_False,SIs_quarter_end_False,SIs_quarter_end_True,SIs_quarter_start_False,SIs_quarter_start_True,SIs_year_end_False,SIs_year_end_True,SIs_year_start_False,SIs_year_start_True
0,HKQuantityTypeIdentifierDietaryWater,709.7650,6.9,2015,7,29,17,4,198,1.437091e+09,...,1,1,1,0,1,0,1,0,1,0
1,HKQuantityTypeIdentifierBodyMassIndex,23.4358,6.9,2015,7,29,17,4,198,1.437091e+09,...,1,1,1,0,1,0,1,0,1,0
2,HKQuantityTypeIdentifierBodyMass,187.5010,6.9,2015,7,29,17,4,198,1.437091e+09,...,1,1,1,0,1,0,1,0,1,0
3,HKQuantityTypeIdentifierStepCount,81.0000,6.9,2015,7,29,17,4,198,1.437091e+09,...,1,1,1,0,1,0,1,0,1,0
4,HKQuantityTypeIdentifierStepCount,10.0000,6.9,2015,7,29,17,4,198,1.437091e+09,...,1,1,1,0,1,0,1,0,1,0


In [43]:
df_encoded_data = pd.get_dummies(df_encoded_data, columns=['type'])

In [44]:
df_encoded_data.head()

,value,sleep_hours,SYear,SMonth,SWeek,SDay,SDayofweek,SDayofyear,SElapsed,StimeYear,...,type_HKQuantityTypeIdentifierSixMinuteWalkTestDistance,type_HKQuantityTypeIdentifierStairAscentSpeed,type_HKQuantityTypeIdentifierStairDescentSpeed,type_HKQuantityTypeIdentifierStepCount,type_HKQuantityTypeIdentifierVO2Max,type_HKQuantityTypeIdentifierWalkingAsymmetryPercentage,type_HKQuantityTypeIdentifierWalkingDoubleSupportPercentage,type_HKQuantityTypeIdentifierWalkingHeartRateAverage,type_HKQuantityTypeIdentifierWalkingSpeed,type_HKQuantityTypeIdentifierWalkingStepLength
0,709.7650,6.9,2015,7,29,17,4,198,1.437091e+09,2023,...,0,0,0,0,0,0,0,0,0,0
1,23.4358,6.9,2015,7,29,17,4,198,1.437091e+09,2023,...,0,0,0,0,0,0,0,0,0,0
2,187.5010,6.9,2015,7,29,17,4,198,1.437091e+09,2023,...,0,0,0,0,0,0,0,0,0,0
3,81.0000,6.9,2015,7,29,17,4,198,1.437091e+09,2023,...,0,0,0,1,0,0,0,0,0,0
4,10.0000,6.9,2015,7,29,17,4,198,1.437091e+09,2023,...,0,0,0,1,0,0,0,0,0,0


In [51]:
df_encoded_data.drop('SElapsed')

KeyError: "['SElapsed'] not found in axis"

In [52]:
df_encoded_data = df_encoded_data.dropna()

In [53]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [54]:
# split the data into features and target
X = df_encoded_data.drop('sleep_hours', axis=1)
y = df_encoded_data['sleep_hours']



In [55]:
# create a random forest regression model
model = RandomForestRegressor()



In [56]:
# train the model
model.fit(X, y)



RandomForestRegressor()

In [57]:
# make predictions on the test data
y_pred = model.predict(X_test)


NameError: name 'X_test' is not defined

In [ ]:

# calculate the RMSE loss
rmse = mean_squared_error(y_test, y_pred, squared=False)


In [ ]:

# print the RMSE loss
print('RMSE:', rmse)

In [61]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [59]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [66]:
 #create an XGBoost regression model
model = xgb.XGBRegressor(max_depth=3, learning_rate=0.1, n_estimators=500, objective='reg:squarederror', booster='gbtree')


In [67]:
model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=500, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [68]:
# make predictions on the test data
y_pred = model.predict(X_test)

# calculate the RMSE loss
rmse = mean_squared_error(y_test, y_pred, squared=False)

In [69]:
# print the RMSE loss
print('RMSE:', rmse)

RMSE: 0.458212422692397
